<a href="https://colab.research.google.com/github/TasnubaTiha33/Senior-Design_Speech-Emotion/blob/main/Wav2vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate
!pip install soundfile
import torch
import json
import librosa
import pandas as pd
import numpy as np
import warnings
import random
import os
import argparse
import glob
from sklearn.metrics import recall_score, accuracy_score, precision_recall_fscore_support, confusion_matrix
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset, DatasetDict, Audio, load_dataset
from transformers import (
    Wav2Vec2FeatureExtractor,
    AutoModelForAudioClassification,
    Trainer,
    TrainingArguments,
    Wav2Vec2Processor
)
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import notebook_login

notebook_login()

**Loading Base wav2vec2 model**

In [3]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

In [4]:
%%capture
!git clone https://github.com/aalto-speech/mtkd4ser.git

In [5]:
%%capture
!pip install gdown

In [7]:
%%capture
!gdown --id 1EZTvcHo2x3cim4uSTkfMHoagipfQtLak
import time
time.sleep(5)

!unzip /content/FESC_CaFE.zip

In [8]:
import json

# Paths to train and test JSON files
# train_json_path = os.path.join(extract_path, 'train.json')
# test_json_path = os.path.join(extract_path, 'test.json')

# Load JSON files
with open("/content/CaFE/CaFE_splits/train.json",'r') as f:
    train_data = json.load(f)

with open("/content/CaFE/CaFE_splits/test.json",'r') as f:
    test_data = json.load(f)

In [9]:
# train_data

In [10]:
train_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in train_data.items()
]
train_dataset = Dataset.from_list(train_samples_list)

In [11]:
test_samples_list = [
    {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
]
test_dataset = Dataset.from_list(test_samples_list)

In [ ]:
# dev_samples_list = [
#     {"id": sample_id, **sample_data} for sample_id, sample_data in test_data.items()
# ]
# dev_dataset = Dataset.from_list(dev_samples_list)

In [12]:
dataset = DatasetDict({
    'train' : train_dataset,
    'test' : test_dataset,
    'dev' : test_dataset
})
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo'],
        num_rows: 156
    })
})

In [ ]:
# for i in range(5):
#     print(f"Sample {i+1} wav path:", train_dataset[i]['file_path'])

In [13]:
def add_base_path(example):
    example['wav'] = '/content/CaFE/' + example['wav']  # Adjust column name if it's not 'wav'
    return example

dataset = dataset.map(add_base_path, batched=False)
dataset['train'][0]


Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

{'id': '09-J-1-4',
 'wav': '/content/CaFE/CaFE_48k/Happiness/Faible/09-J-1-4.wav',
 'length': 3.5731875,
 'emo': 'happiness'}

In [14]:
import librosa
def load_audio(example):
  audio_array, sample_rate = librosa.load(example['wav'], sr=16000)
  example["audio_array"] = audio_array
  example["sample_rate"] = sample_rate
  return example

In [15]:
dataset = dataset.map(load_audio, batched=False)
dataset

Map:   0%|          | 0/780 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

Map:   0%|          | 0/156 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 780
    })
    test: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
    dev: Dataset({
        features: ['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'],
        num_rows: 156
    })
})

In [16]:
dataset['train'][0].keys()

dict_keys(['id', 'wav', 'length', 'emo', 'audio_array', 'sample_rate'])

In [17]:
!pip install tensorboard
from torch.utils.tensorboard import SummaryWriter
!pip install tqdm
from tqdm import tqdm

In [18]:
# Metrics functions
def calculate_unweighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='macro')

def calculate_weighted_recall(y_true, y_pred):
    return recall_score(y_true, y_pred, average='weighted')

def calculate_accuracy(y_true, y_pred):
    return accuracy_score(y_true, y_pred)

# Collate function for DataLoader
def collate_fn(batch):
    batch = sorted(batch, key=lambda x: len(x["input_values"]), reverse=True)
    inputs = [torch.tensor(example["input_values"]) for example in batch]
    labels = [int(example["label"]) for example in batch]
    inputs = pad_sequence(inputs, batch_first=True)
    return {"input_values": inputs, "label": torch.tensor(labels)}

# Example helper function to update paths
def update_cafe_path(path):
    return "/content/CaFE/" + path  # Adjust as necessary for your file path

In [19]:
def is_common(label):
    return label in ['anger', 'happiness', 'neutral', 'sadness']

def to_label(label, label2id):
    return label2id[label]

def cafe(session=None):
    all_files = glob.glob("/content/CaFE/CaFE_splits/*")
    cafe_df = pd.DataFrame()

    for file in all_files:
        with open(file, "r") as f:
            data = json.load(f)
            df = pd.DataFrame.from_dict(data, orient="index").reset_index()
            df["set"] = [file.split("/")[-1].split(".")[0] for _ in range(len(df))]
            df = df.rename(columns={'wav': 'audio', 'emo': 'label'})
            df["label_flag"] = df["label"].apply(is_common)
            cafe_df = pd.concat([cafe_df, df], axis=0)

    cafe_df = cafe_df.loc[cafe_df["label_flag"] == 1].reset_index(drop=True)
    cafe_df_train = cafe_df.loc[cafe_df["set"] != "test"][["audio", "label"]]
    cafe_df_test = cafe_df.loc[cafe_df["set"] == "test"][["audio", "label"]]

    cafe_df_train["audio"] = cafe_df_train["audio"].apply(update_cafe_path).values
    cafe_df_test["audio"] = cafe_df_test["audio"].apply(update_cafe_path).values

    labels = ['anger', 'happiness', 'neutral', 'sadness']
    label2id, id2label = {label: str(i) for i, label in enumerate(labels)}, {str(i): label for i, label in enumerate(labels)}

    train_df, test_df = cafe_df_train, cafe_df_test
    train_df["label"] = train_df["label"].apply(to_label, args=(label2id,)).values
    test_df["label"] = test_df["label"].apply(to_label, args=(label2id,)).values

    train_audio_data = Dataset.from_pandas(train_df[['audio', 'label']]).cast_column("audio", Audio(sampling_rate=16_000))
    test_audio_data = Dataset.from_pandas(test_df[['audio', 'label']]).cast_column("audio", Audio(sampling_rate=16_000))

    return label2id, id2label, DatasetDict({'train': train_audio_data, 'test': test_audio_data, 'dev': test_audio_data})

def preprocess_function(examples, feature_extractor):
    max_duration = 20.0
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=int(feature_extractor.sampling_rate * max_duration), padding=True, truncation=True)
    return inputs

In [32]:
def model_ft(label2id, id2label, num_classes, device="cpu"):
    model_name = "facebook/wav2vec2-base"
    model = AutoModelForAudioClassification.from_pretrained(model_name, num_labels=num_classes, label2id=label2id, id2label=id2label)
    model.to(device)
    return model

def pipeline_ft(model, train_loader, valid_loader, test_loader, hyperparam, device):
    SESSION = hyperparam['SESSION']
    LEARNING_RATE = hyperparam['LEARNING_RATE']
    TRAINING = hyperparam['TRAINING']
    N_EPOCHS = hyperparam['N_EPOCHS']
    LINGUALITY = hyperparam['LINGUALITY']
    LANGUAGE = hyperparam['LANGUAGE']

    file_path = f"/content/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}.pth"
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_fn = torch.nn.CrossEntropyLoss()

    # Load checkpoint if exists
    epoch = 1
    if os.path.exists(file_path):
        checkpoint = torch.load(file_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        total_train_loss = checkpoint.get('training loss', 0)
        total_valid_loss = checkpoint.get('validation loss', 0)
        print("Model checkpoint has been loaded")

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=f"/content/tensorboards/FT_{LINGUALITY}_{LANGUAGE}_S{SESSION}")

    # Evaluation-only mode
    if TRAINING == 0:
        test_unweighted_recall, test_weighted_recall, test_accuracy = validation(
            model,
            tqdm(test_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Test", leave=False),
            loss_fn,
            device
        )
        print(f"Epoch {epoch}/{N_EPOCHS}, Test Recall (unweighted): {test_unweighted_recall}, Recall (weighted): {test_weighted_recall}, Accuracy: {test_accuracy}\n")

    else:
        # Training and validation mode
        for epoch in range(epoch, N_EPOCHS+1):
            # Train step - adjust to returned values
            train_unweighted_recall, train_weighted_recall, train_accuracy = train(
                model,
                tqdm(train_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Training", leave=False),
                optimizer,
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Training Recall (unweighted): {train_unweighted_recall}, Recall (weighted): {train_weighted_recall}, Accuracy: {train_accuracy}")

            # Log training metrics
            writer.add_scalar("Accuracy/Train", train_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Train", train_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Train", train_weighted_recall, epoch)

            # Validation step - adjust to returned values
            valid_unweighted_recall, valid_weighted_recall, valid_accuracy = validation(
                model,
                tqdm(valid_loader, desc=f"Epoch {epoch}/{N_EPOCHS}, Validation", leave=False),
                loss_fn,
                device
            )
            print(f"Epoch {epoch}/{N_EPOCHS}, Validation Recall (unweighted): {valid_unweighted_recall}, Recall (weighted): {valid_weighted_recall}, Accuracy: {valid_accuracy}\n")

            # Log validation metrics
            writer.add_scalar("Accuracy/Validation", valid_accuracy, epoch)
            writer.add_scalar("Recall_Unweighted/Validation", valid_unweighted_recall, epoch)
            writer.add_scalar("Recall_Weighted/Validation", valid_weighted_recall, epoch)

            writer.flush()

            # Save model checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'training loss': train_unweighted_recall,  # Adjust as needed
                'validation loss': valid_unweighted_recall  # Adjust as needed
            }, file_path)

            print(f"Model has been saved after epoch: {epoch}\n")

        # Close TensorBoard writer
        writer.close()

def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_train_loss, correct_predictions, total_predictions = 0, 0, 0
    all_actual_labels, all_predictions = [], []

    for batch in train_loader:
        inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)
        all_actual_labels.extend(labels.tolist())
        all_predictions.extend(predicted.tolist())

    unweighted_recall = recall_score(all_actual_labels, all_predictions, average='macro')
    return unweighted_recall, correct_predictions / total_predictions, total_train_loss / len(train_loader)
# Define the validation function
def validation(model, data_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for batch in data_loader:
            inputs, labels = batch['input_values'].to(device), batch['label'].to(device)
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predictions.cpu().numpy())

    unweighted_recall = recall_score(all_labels, all_predictions, average='macro')
    weighted_recall = recall_score(all_labels, all_predictions, average='weighted')
    accuracy = sum(np.array(all_labels) == np.array(all_predictions)) / len(all_labels)

    return unweighted_recall, weighted_recall, accuracy

In [33]:
# Define the Args class with instance variables
class Args:
    def __init__(self):
        self.N_EPOCHS = 10
        self.LEARNING_RATE = 1e-5
        self.BATCH_SIZE = 8
        self.SESSION = 1
        self.TRAINING = 1
        self.PARADIGM = "FT"
        self.LANGUAGE = "FR"
        self.LINGUALITY = "Monolingual"

In [34]:
def main(args_dict):
    random_seed = 42
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(random_seed)

    # Access values using args_dict
    session = args_dict["SESSION"]
    batch_size = args_dict["BATCH_SIZE"]
    learning_rate = args_dict["LEARNING_RATE"]

    # Load the CaFE dataset
    label2id, id2label, ds = cafe(session)

    # Initialize feature extractor
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base")

    # Preprocess dataset
    encoded_audio = ds.map(lambda example: preprocess_function(example, feature_extractor), remove_columns="audio", batched=True)

    # Initialize DataLoaders
    train_loader = DataLoader(encoded_audio["train"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    test_loader = DataLoader(encoded_audio["test"], batch_size=batch_size, collate_fn=collate_fn)
    valid_loader = DataLoader(encoded_audio["dev"], batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # Model and optimizer
    model = model_ft(label2id, id2label, len(label2id), device="cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # Run pipeline
    pipeline_ft(model, train_loader, valid_loader, test_loader, args_dict, device="cuda" if torch.cuda.is_available() else "cpu")

In [35]:
# Create an instance of Args and pass it as a dictionary to main
args = Args()
main(args.__dict__)

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10, Training Recall (unweighted): 0.3479166666666667, Recall (weighted): 0.3976190476190476, Accuracy: 1.31374957426539


...

Epoch 8/10, Validation Recall (unweighted): 0.6458333333333334, Recall (weighted): 0.6190476190476191, Accuracy: 0.6190476190476191

Model has been saved after epoch: 8



Epoch 9/10, Training Recall (unweighted): 0.95, Recall (weighted): 0.95, Accuracy: 0.20851998565331945


Epoch 9/10, Validation Recall (unweighted): 0.6875, Recall (weighted): 0.6785714285714286, Accuracy: 0.6785714285714286

Model has been saved after epoch: 9



Epoch 10/10, Training Recall (unweighted): 0.9604166666666666, Recall (weighted): 0.9642857142857143, Accuracy: 0.16142042901999545


Epoch 10/10, Validation Recall (unweighted): 0.7083333333333334, Recall (weighted): 0.7023809523809523, Accuracy: 0.7023809523809523

Model has been saved after epoch: 10



In [ ]:
# import evaluate
# accuracy = evaluate.load("accuracy")

In [ ]:
# import numpy as np


# def compute_metrics(eval_pred):
#     predictions = np.argmax(eval_pred.predictions, axis=1)
#     return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
# from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer, AutoFeatureExtractor

# num_labels = len(id2label)
# model = AutoModelForAudioClassification.from_pretrained(
#     "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
# )

pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# training_args = TrainingArguments(
#     output_dir="Cafe_model",
#     eval_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=3e-5,
#     per_device_train_batch_size=32,
#     gradient_accumulation_steps=4,
#     per_device_eval_batch_size=32,
#     num_train_epochs=10,
#     warmup_ratio=0.1,
#     logging_steps=10,
#     load_best_model_at_end=True,
#     metric_for_best_model="accuracy",
#     # push_to_hub=True,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
#     tokenizer=processor.feature_extractor,
#     compute_metrics=compute_metrics,
# )

# trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,1.269237,0.404762
2,1.157400,1.137223,0.559524
4,1.157400,1.050499,0.619048
6,0.985400,1.010320,0.583333
8,0.868000,1.006469,0.619048


TrainOutput(global_step=30, training_loss=1.0035688400268554, metrics={'train_runtime': 240.2345, 'train_samples_per_second': 17.483, 'train_steps_per_second': 0.125, 'total_flos': 1.916717986393703e+17, 'train_loss': 1.0035688400268554, 'epoch': 8.571428571428571})

In [ ]:
# # Define the model name for the Hugging Face Hub
# model_name = "Tasnuba2012848/Cafe_model"

# # Push the entire Trainer (including model, optimizer, scheduler, etc.) to the Hugging Face Hub
# trainer.push_to_hub(model_name)

# # Alternatively, if you specifically want to save and push the model and processor separately
# from transformers import Wav2Vec2Processor

# # Save both the model and processor locally
# model.save_pretrained("local_model_dir")
# processor.save_pretrained("local_model_dir")

# # Push the model and processor to the Hugging Face Hub (if you want separate pushes)
# model.push_to_hub(model_name)
# processor.push_to_hub(model_name)

events.out.tfevents.1730727098.4e8890c9db33.4363.0:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Tasnuba2012848/Cafe_model/commit/e301f68b5b14ede9b1a1e5d21dc29a302fbb4c07', commit_message='Upload processor', commit_description='', oid='e301f68b5b14ede9b1a1e5d21dc29a302fbb4c07', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
# from datasets import load_dataset
# import torch
# from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# # Load model and processor from the Hugging Face Hub
# model_name = "Tasnuba2012848/Cafe_model"
# model = Wav2Vec2ForCTC.from_pretrained(model_name)
# processor = Wav2Vec2Processor.from_pretrained(model_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at Tasnuba2012848/Cafe_model and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# pip install soundfile

In [ ]:
# print("Keys in test_dataset:", test_dataset.column_names)

Keys in test_dataset: ['id', 'length', 'emo', 'input_values', 'labels']


In [ ]:
# # Import necessary libraries
# import numpy as np
# from sklearn.metrics import precision_recall_fscore_support
# from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor

# # Load model and processor from the Hugging Face Hub
# model_name = "Tasnuba2012848/Cafe_model"
# model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)
# processor = Wav2Vec2Processor.from_pretrained(model_name)

# # Assuming the dataset is already created as before
# test_dataset = dataset["test"]

# # Run predictions on the test dataset
# predictions = trainer.predict(test_dataset)
# predicted_labels = np.argmax(predictions.predictions, axis=1)
# true_labels = predictions.label_ids

# # Function to calculate weighted and unweighted recall
# def calculate_recall_metrics(true_labels, predicted_labels):
#     # Compute per-class precision, recall, F1
#     _, recall_per_class, _, _ = precision_recall_fscore_support(
#         true_labels, predicted_labels, average=None, zero_division=0
#     )

#     # Compute weighted and unweighted (macro) recall
#     weighted_recall = precision_recall_fscore_support(
#         true_labels, predicted_labels, average='weighted', zero_division=0
#     )[1]  # Index 1 is recall

#     unweighted_recall = recall_per_class.mean()  # Mean recall for unweighted (macro)

#     return {
#         "weighted_recall": weighted_recall,
#         "unweighted_recall": unweighted_recall
#     }

# # Display the recall metrics
# recall_metrics = calculate_recall_metrics(true_labels, predicted_labels)
# print("Weighted Recall:", recall_metrics["weighted_recall"])
# print("Unweighted Recall:", recall_metrics["unweighted_recall"])

Weighted Recall: 0.6190476190476191
Unweighted Recall: 0.5416666666666666


In [ ]:
# import logging
# from transformers import TrainerCallback

# # Set up logging
# logging.basicConfig(level=logging.INFO)
# logger = logging.getLogger(__name__)

# class PrintMetricsCallback(TrainerCallback):
#     def on_epoch_end(self, args, state, control, metrics=None, **kwargs):
#         if metrics is not None:
#             # Extract training and evaluation metrics
#             train_metrics = {k: v for k, v in metrics.items() if "train" in k}
#             eval_metrics = {k: v for k, v in metrics.items() if "eval" in k}

#             # Log the training and evaluation metrics
#             logger.info(f"Training metrics after epoch {state.epoch}: {train_metrics}")
#             logger.info(f"Evaluation metrics after epoch {state.epoch}: {eval_metrics}")

# # Instantiate the Trainer with the custom callback
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset["train"],
#     eval_dataset=dataset["test"],
#     tokenizer=processor.feature_extractor,
#     compute_metrics=compute_metrics,
#     callbacks=[PrintMetricsCallback()]  # Add your custom callback here
# )

# # Start training
# trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,1.213218,0.511905
2,0.968700,1.052150,0.583333
4,0.968700,0.954152,0.619048
6,0.790800,0.940440,0.630952
8,0.649800,0.912081,0.654762


TrainOutput(global_step=30, training_loss=0.8030888080596924, metrics={'train_runtime': 207.7254, 'train_samples_per_second': 20.219, 'train_steps_per_second': 0.144, 'total_flos': 1.916717986393703e+17, 'train_loss': 0.8030888080596924, 'epoch': 8.571428571428571})